# Template extraction #

Parse text documents which are based on a template and stored in an excel document

Install dependencies and import the nlp_datau package

In [ ]:
import yaml
!pip install -r ./../requirements.txt
import sys
sys.path.append("./../")


## 1. Extract substring of substring ##

Set variables

In [ ]:
path_excel_sheet_in = '/home/jovyan/work/E/Sander/Outcomes until 20 May incl verslagen.xlsx'
path_excel_sheet_out = '/home/jovyan/work/E/Sander/Outcomes until 20 May incl verslagen_incidental findings_co-rads.xlsx'

# path_excel_sheet_in = './../resources/data_example/standaardverslag_covid.xlsx'
# path_excel_sheet_out = './../resources/data_example/standaardverslag_covid_out.xlsx'

column_source = 'text'
column_target_conclusion = 'conclusion'
column_target_incidental_findings = 'incidental_findings'
column_target_missing_incidental_findings = 'missing_incidental_findings'
column_corads = 'corads'

regex_conclusion = r"(CONCLUSIE:|Conclusie:)(.*?)(DISCLAIMER|Met collegiale groet)"
regex_incidental_findings = r"(?<=Nevenbevindingen:)(.*?)(?=(DISCLAIMER|Met collegiale groet))"

import pandas
from IPython.core.display import display
df = pandas.read_excel(path_excel_sheet_in, header=0)
display(df)

Extract findings

In [ ]:
from nlp_datau.string_utils import select_regex, regex_missing, normalize_regex

df[column_target_conclusion] = df[column_source].apply(lambda x:select_regex(x, regex=regex_conclusion))
df[column_target_incidental_findings] = df[column_target_conclusion].apply(lambda x:select_regex(x, regex=regex_incidental_findings))
df[column_target_missing_incidental_findings] = df[column_target_conclusion].apply(lambda x:regex_missing(x, regex=regex_incidental_findings))


Extract corads

In [ ]:
corads_config = './../resources/config/pycontextnlp_tnm_targets_corads_nl.yml'

stream = open(corads_config, "r")
targets = yaml.load_all(stream)
regex_list = []
for target in targets:
    regex_list.append(target)

df[column_corads] = df[column_target_conclusion].apply(lambda x:normalize_regex(x, targets=regex_list))
display(df)

Run and write to out file

In [ ]:
df.to_excel(path_excel_sheet_out)